In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '18'
plt.rcParams['figure.dpi'] = 125
plt.rcParams['figure.facecolor'] = 'white'

import numpy as np
import xarray as xr
import nctoolkit as nc
import datetime

import os, sys

module_path = os.path.abspath(os.path.join('/Users/gilaverbuch/dev/pysst/'))
if module_path not in sys.path:
    sys.path.appendp(module_path)
from src.pysst.pysst import load_data

# from pysst import load_data

In [ ]:
date_i = '2019-02-10T15:00'
date_f = '2019-02-11T00:00'

date_i = np.datetime64(date_i)
date_f = np.datetime64(date_f)

sst = load_data(date_i, date_f, merge=True)

In [ ]:
sst

In [ ]:
t=2
sst.isel(time=t).mcsst.plot(cmap='gnuplot2', clim=(5,35))

In [ ]:
sst.isel(time=t).mcsst_source1.plot(cmap='gnuplot2', clim=(5,35))

In [ ]:
sst.isel(time=t).mcsst_source2.plot(cmap='gnuplot2', clim=(5,35))